In [ ]:
import math
import pygame

# ---------------- Config ----------------
SCREEN_W, SCREEN_H = 800, 600
ROOM_HEIGHT = 150                 # world pixels per room
ROOM_WIDTH = 150
WALL_THICK = 5
DOOR_SIZE = 40
PLAYER_RADIUS = 8
PLAYER_SPEED = 250
BG_COLOR = (15, 15, 18)
FLOOR_COLOR = (25, 25, 28)
WALL_COLOR = (200, 200, 200)
PLAYER_COLOR = (100, 200, 255)
FPS = 60
NUMBER_ROOMS = 9


# --------------- Helpers ----------------
def clamp(v, lo, hi):
    return max(lo, min(v, hi))

def world_to_chunk(x, y):
    """Return integer (rx, ry) chunk coordinates for a world position."""
    rx = math.floor(x / ROOM_SIZE)
    ry = math.floor(y / ROOM_SIZE)
    return rx, ry

def chunk_rect(rx, ry):
    """Return pygame.Rect in world coords for a room chunk."""
    return pygame.Rect(rx * ROOM_SIZE, ry * ROOM_SIZE, ROOM_SIZE, ROOM_SIZE)

# --------------- Room / World -----------

class Room:
    """
    A square room with 4 walls and a door gap in the middle of each wall.
    We precompute *wall rectangles* (in world coordinates) with the door cut out.
    """
    def __init__(self, rx, ry):
        self.rx = rx
        self.ry = ry
        self.rect = chunk_rect(rx, ry)
        self.wall_rects = self._build_walls()

    def _build_walls(self):
        r = self.rect
        t = WALL_THICK
        door = DOOR_SIZE

        walls = []

        # Helpers to add wall segments
        def add_rect(x, y, w, h):
            walls.append(pygame.Rect(x, y, w, h))

        # TOP wall (split around a centered door)
        top_y = r.top
        door_left = r.centerx - door // 2
        add_rect(r.left, top_y, door_left - r.left, t)  # left segment
        add_rect(door_left + door, top_y, r.right - (door_left + door), t)  # right segment

        # BOTTOM wall
        bottom_y = r.bottom - t
        add_rect(r.left, bottom_y, door_left - r.left, t)
        add_rect(door_left + door, bottom_y, r.right - (door_left + door), t)

        # LEFT wall
        left_x = r.left
        door_top = r.centery - door // 2
        add_rect(left_x, r.top, t, door_top - r.top)
        add_rect(left_x, door_top + door, t, r.bottom - (door_top + door))

        # RIGHT wall
        right_x = r.right - t
        add_rect(right_x, r.top, t, door_top - r.top)
        add_rect(right_x, door_top + door, t, r.bottom - (door_top + door))

        return walls

class World:
    def __init__(self):
        # map[(rx, ry)] = Room
        self.rooms = {}

    def ensure_rooms_around(self, center_rx, center_ry, radius=LOAD_RADIUS_ROOMS):
        """Generate rooms within radius if missing; unload far rooms."""
        needed = set()
        for dx in range(-radius, radius + 1):
            for dy in range(-radius, radius + 1):
                needed.add((center_rx + dx, center_ry + dy))

        # Load
        for key in needed:
            if key not in self.rooms:
                self.rooms[key] = Room(*key)

        # Unload far
        to_delete = [k for k in self.rooms.keys()
                     if abs(k[0] - center_rx) > radius or abs(k[1] - center_ry) > radius]
        for k in to_delete:
            del self.rooms[k]

    def visible_rooms(self, cam_rect):
        """Return rooms whose rect intersects the camera + small padding."""
        out = []
        for room in self.rooms.values():
            if room.rect.colliderect(cam_rect.inflate(200, 200)):
                out.append(room)
        return out

# --------------- Physics ----------------

def move_and_collide(player_rect, vx, vy, walls, dt):
    """
    Axis-aligned collision resolution for a circle approximated by its bounding rect.
    """
    # Move X
    player_rect.x += int(vx * dt)
    for w in walls:
        if player_rect.colliderect(w):
            if vx > 0:
                player_rect.right = w.left
            elif vx < 0:
                player_rect.left = w.right

    # Move Y
    player_rect.y += int(vy * dt)
    for w in walls:
        if player_rect.colliderect(w):
            if vy > 0:
                player_rect.bottom = w.top
            elif vy < 0:
                player_rect.top = w.bottom

    return player_rect

# --------------- Main -------------------

def main():
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_W, SCREEN_H))
    clock = pygame.time.Clock()
    pygame.display.set_caption("Rooms with Doors / Chunked Loading")

    world = World()

    # Start player in the origin room (0,0) center
    start_rx, start_ry = 100,200
    start_room = chunk_rect(start_rx, start_ry)
    x = start_room.centerx
    y = start_room.centery

    running = True
    while running:
        dt = clock.tick(FPS) / 1000.0

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # -------- Input --------
        keys = pygame.key.get_pressed()
        dx = dy = 0
        if keys[pygame.K_a] or keys[pygame.K_LEFT]:
            dx -= 1
        if keys[pygame.K_d] or keys[pygame.K_RIGHT]:
            dx += 1
        if keys[pygame.K_w] or keys[pygame.K_UP]:
            dy -= 1
        if keys[pygame.K_s] or keys[pygame.K_DOWN]:
            dy += 1
        if dx and dy:
            inv = 0.70710678
            dx *= inv
            dy *= inv

        # -------- World / Loading --------
        rx, ry = world_to_chunk(x, y)
        world.ensure_rooms_around(rx, ry)

        # Gather nearby wall rects for collision (only rooms near player)
        walls = []
        for rr in world.rooms.values():
            # Only collect walls for rooms within LOAD_RADIUS_ROOMS + 1 to be safe
            if abs(rr.rx - rx) <= LOAD_RADIUS_ROOMS + 1 and abs(rr.ry - ry) <= LOAD_RADIUS_ROOMS + 1:
                walls.extend(rr.wall_rects)

        # -------- Physics --------
        player_rect = pygame.Rect(0, 0, PLAYER_RADIUS * 2, PLAYER_RADIUS * 2)
        player_rect.center = (int(x), int(y))
        player_rect = move_and_collide(player_rect,
                                       dx * PLAYER_SPEED,
                                       dy * PLAYER_SPEED,
                                       walls,
                                       dt)
        x, y = player_rect.center

        # -------- Camera --------
        cam_x = int(x - SCREEN_W / 2)
        cam_y = int(y - SCREEN_H / 2)
        cam_rect = pygame.Rect(cam_x, cam_y, SCREEN_W, SCREEN_H)

        # -------- Draw --------
        screen.fill(BG_COLOR)

        # Draw floors (just a solid rect) + walls for visible rooms
        for room in world.visible_rooms(cam_rect):
            # Floor
            floor_rect = room.rect.move(-cam_x, -cam_y)
            pygame.draw.rect(screen, FLOOR_COLOR, floor_rect)

            # Walls
            for w in room.wall_rects:
                draw_rect = w.move(-cam_x, -cam_y)
                pygame.draw.rect(screen, WALL_COLOR, draw_rect)

        # Player
        pygame.draw.circle(screen, PLAYER_COLOR,
                           (int(x - cam_x), int(y - cam_y)),
                           PLAYER_RADIUS)

        pygame.display.set_caption(f"Chunked Rooms | FPS: {clock.get_fps():.0f} | Rooms loaded: {len(world.rooms)}")
        pygame.display.flip()

    pygame.quit()

if __name__ == "__main__":
    main()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html
